In [9]:
import pandas as pd
import numpy as np
import unidecode
import requests
from tqdm import tqdm

In [23]:
def is_url_image(image_url):
        image_formats = ("image/png", "image/jpeg", "image/jpg")
        r = requests.head(image_url)
        if r.headers["content-type"] in image_formats:
            return True
        return False
    
def try_get_new_images(nome):
    
    unaccented_nome = unidecode.unidecode(nome).upper()
    url = 'https://content.fantacalcio.it/web/campioncini/card/'+ unaccented_nome + '.png?v=12'
    
    if is_url_image(url):
        return url    
    else:
        return 'https://content.fantacalcio.it/web/campioncini/card/NO-CAMPIONCINO.png?v=12'
    

In [24]:
stats = pd.read_csv('../dataset/stat_info^(Stagione 2021_22).csv')
listone = pd.read_excel('../dataset/quotazioni^(Stagione 2022_23).xlsx',skiprows=[0])

In [25]:
listone = listone[['Id','R','Nome','Squadra']]

In [26]:
final_listone_stats = listone.merge(stats.drop(columns=['Nome','Squadra','Ruolo']), left_on='Id', right_on='Id', how='left').rename(columns={'R':'Ruolo'}).drop(columns='Unnamed: 0')

In [27]:
new_img_list = []
for idx, row in tqdm(final_listone_stats.iterrows()):
    if row.img is np.nan:
        img = try_get_new_images(row.Nome)
        new_img_list.append({'Id':row.Id,'img':img})

514it [00:17, 29.97it/s]


In [28]:
new_img_df = pd.DataFrame(new_img_list)
for idx, row in tqdm(final_listone_stats.iterrows()):
    if row.Id in new_img_df.Id.unique():
        final_listone_stats.loc[final_listone_stats.Id == row.Id,'img'] = new_img_df.loc[new_img_df.Id==row.Id,'img'].values

514it [00:00, 1778.56it/s]


In [ ]:
final_listone_stats.to_csv('../dataset/final_listone^(Stagione 2022_23).csv')